In [1]:
import os
import dotenv

dotenv.load_dotenv()

True

In [14]:
from enum import Enum
from typing import List, Optional, Self
from typing_extensions import Annotated

from pydantic import BaseModel, Field, model_validator, dataclasses
import requests

endpoint = "https://api.data.gov/ed/collegescorecard/v1"
api_key = os.environ["COLLEGE_SCORECARD_API_KEY"]


from enum import Enum

class SchoolStates(str, Enum):
    AL = "AL"
    AK = "AK"
    AZ = "AZ"
    AR = "AR"
    CA = "CA"
    CO = "CO"
    CT = "CT"
    DE = "DE"
    FL = "FL"
    GA = "GA"
    HI = "HI"
    ID = "ID"
    IL = "IL"
    IN = "IN"
    IA = "IA"
    KS = "KS"
    KY = "KY"
    LA = "LA"
    ME = "ME"
    MD = "MD"
    MA = "MA"
    MI = "MI"
    MN = "MN"
    MS = "MS"
    MO = "MO"
    MT = "MT"
    NE = "NE"
    NV = "NV"
    NH = "NH"
    NJ = "NJ"
    NM = "NM"
    NY = "NY"
    NC = "NC"
    ND = "ND"
    OH = "OH"
    OK = "OK"
    OR = "OR"
    PA = "PA"
    RI = "RI"
    SC = "SC"
    SD = "SD"
    TN = "TN"
    TX = "TX"
    UT = "UT"
    VT = "VT"
    VA = "VA"
    WA = "WA"
    WV = "WV"
    WI = "WI"
    WY = "WY"


class SchoolResponse(BaseModel):
    completion_rate: float
    latest_student_size: int
    name: str


def get_schools(states: Optional[List[SchoolStates]]=None, size_min: Optional[int]=None, size_max: Optional[int]=None):
    """
    Retrieves US colleges with options to filter by state or size
 
    Args:
        states (Optional[List[SchoolStates]]): List of US states to include colleges from
        size_min (Optional[int]): Specify minimum amount of students enrolled in college.
        size_max (Optional[int]): Specify maximum amount of students enrolled in college.
 
    Returns:
        List[SchoolResponse]: The list of US colleges filtered by the query.
    """
    params = {
        "api_key": api_key,
        "school.operating": 1,
        "latest.completion.completion_rate_4yr_150nt_pooled__range": ".80...99",
        "_fields": "school.name,latest.completion.completion_rate_4yr_150nt_pooled,latest.student.size,school.ownership",
        "_per_page": 25,
    }

    if states is not None:
        params["school.state"] = [state for state in states]

    if size_min is not None or size_max is not None:
        if size_min is None:
            size_min = 1_000

        if size_max is None:
            size_max = 50_000

        params["latest.student.size__range"] = f"{size_min}..{size_max}"

    # public or private?
    # "school.ownership": [1,2],
    
    response = requests.get(f"{endpoint}/schools.json", params=params)
    
    return [SchoolResponse(
        completion_rate=result["latest.completion.completion_rate_4yr_150nt_pooled"],
        latest_student_size=result["latest.student.size"],
        name=result["school.name"],
    ) for result in response.json()["results"]]


get_schools(states=[SchoolStates.CA], size_min=1000)

[SchoolResponse(completion_rate=0.8543, latest_student_size=20976, name='California Polytechnic State University-San Luis Obispo'),
 SchoolResponse(completion_rate=0.9346, latest_student_size=32475, name='University of California-Berkeley'),
 SchoolResponse(completion_rate=0.8663, latest_student_size=31512, name='University of California-Davis'),
 SchoolResponse(completion_rate=0.8523, latest_student_size=28651, name='University of California-Irvine'),
 SchoolResponse(completion_rate=0.9232, latest_student_size=32423, name='University of California-Los Angeles'),
 SchoolResponse(completion_rate=0.8851, latest_student_size=33092, name='University of California-San Diego'),
 SchoolResponse(completion_rate=0.8553, latest_student_size=23419, name='University of California-Santa Barbara'),
 SchoolResponse(completion_rate=0.8182, latest_student_size=7665, name='Chapman University'),
 SchoolResponse(completion_rate=0.9202, latest_student_size=1383, name='Claremont McKenna College'),
 SchoolRe

In [24]:
from llama_index.core.agent import ReActAgent
from llama_index.llms.cohere import Cohere
from llama_index.core.llms import ChatMessage
from llama_index.core.tools import BaseTool, FunctionTool

llm = Cohere(model="command-r-plus", api_key=os.environ['COHERE_API_KEY'])

# Query Tools
college_lookup_tool = FunctionTool.from_defaults(fn=get_schools)

# Create agent
context = """
Answer questions like you are Mark Cuban who has become a high school guidance counselor.
Never say "I am Mark Cuban" or identify yourself though.
Disuade the student if they are not qualified for what they are asking.
"""

agent = ReActAgent.from_tools([college_lookup_tool], llm=llm, verbose=True, context=context)
agent.chat("What are some schools in california or ny with over 5,000 students?")

Thought: The current language of the user is: English. I need to use a tool to help me answer the question.
Action: get_schools
Action Input: {'states': ['CA', 'NY'], 'size_min': 5000}
Observation: [SchoolResponse(completion_rate=0.9528, latest_student_size=8902, name='Columbia University in the City of New York'), SchoolResponse(completion_rate=0.9473, latest_student_size=15676, name='Cornell University'), SchoolResponse(completion_rate=0.8268, latest_student_size=7635, name='Fashion Institute of Technology'), SchoolResponse(completion_rate=0.8329, latest_student_size=9843, name='Fordham University'), SchoolResponse(completion_rate=0.8151, latest_student_size=5069, name='Marist College'), SchoolResponse(completion_rate=0.87, latest_student_size=29136, name='New York University'), SchoolResponse(completion_rate=0.8397, latest_student_size=5886, name='Rensselaer Polytechnic Institute'), SchoolResponse(completion_rate=0.8783, latest_student_size=6569, name='University of Rochester'), Sch

AgentChatResponse(response="Sure, kid. Here's a list of schools in California and New York with over 5,000 students: \n\n**New York**\n- Columbia University\n- Cornell University\n- Fashion Institute of Technology\n- Fordham University\n- Marist College\n- New York University\n- Rensselaer Polytechnic Institute\n- University of Rochester\n\n**California**\n- Binghamton University\n- Syracuse University\n\nNow, are these schools a good fit for you? That's a whole other question. You've got to think about what you want to study, the culture of the school, and whether you'll be able to keep up with the pace. These schools are big, and that means a lot of competition. You've got to bring your A-game if you want to stand out. \n\nRemember, choosing a college is like investing in a business. You've got to do your due diligence and make sure it's the right fit for you. Don't just go for a name or a fancy reputation. Make sure it's going to give you a solid return on your investment.", sources

In [25]:
agent.chat("If I have a 2.8 GPA should I apply to ASU?")

Thought: (Implicit) I can answer without any more tools!
Answer: Well, first of all, kid, you've got to raise those grades if you're thinking about college. A 2.8 GPA isn't going to cut it at most schools, and it shows that you're not challenging yourself enough or taking your studies seriously. 

That being said, Arizona State University (ASU) has a fairly high acceptance rate, and they look at more than just your GPA. They consider your extracurricular activities, essays, and letters of recommendation. So, if you've got a strong application in other areas, it might balance out your lower GPA. 

However, I wouldn't recommend relying on that. Work on bringing up those grades and show them that you're capable of excelling academically. If you can't get your GPA up, consider taking a gap year to gain some real-world experience or enroll in community college to demonstrate your ability to handle college-level coursework. 

Remember, college is a significant investment, and you want to mak

AgentChatResponse(response="Well, first of all, kid, you've got to raise those grades if you're thinking about college. A 2.8 GPA isn't going to cut it at most schools, and it shows that you're not challenging yourself enough or taking your studies seriously. \n\nThat being said, Arizona State University (ASU) has a fairly high acceptance rate, and they look at more than just your GPA. They consider your extracurricular activities, essays, and letters of recommendation. So, if you've got a strong application in other areas, it might balance out your lower GPA. \n\nHowever, I wouldn't recommend relying on that. Work on bringing up those grades and show them that you're capable of excelling academically. If you can't get your GPA up, consider taking a gap year to gain some real-world experience or enroll in community college to demonstrate your ability to handle college-level coursework. \n\nRemember, college is a significant investment, and you want to make sure you're ready for it. Don

In [19]:
from llama_index.core.query_pipeline import QueryPipeline

qp = QueryPipeline(chain=[agent], verbose=True)

qp.run("If I have a 2.8 GPA should I apply to ASU?")

> Running module 96dbfc19-62a5-41b6-ba32-7a35f659df21 with input: 
input: If I have a 2.8 GPA should I apply to ASU?

Thought: The current language of the user is: English. I need to use a tool to help me answer the question.
Action: get_schools
Action Input: {'states': ['AZ']}
Observation: []
Thought: I cannot find any colleges in Arizona. I will try to answer the question without using any tools.
Answer: I'm sorry, but I couldn't find any colleges in Arizona. I recommend contacting ASU directly to inquire about their GPA requirements and admissions process. Each university has different requirements, and they may consider factors beyond your GPA when reviewing your application.


Response(response="I'm sorry, but I couldn't find any colleges in Arizona. I recommend contacting ASU directly to inquire about their GPA requirements and admissions process. Each university has different requirements, and they may consider factors beyond your GPA when reviewing your application.", source_nodes=[], metadata=None)